This series of notebooks is a Tensorflow implementation of RANZCR 1st Place Solution by [@haqishen](https://www.kaggle.com/haqishen). The original notebooks are the following three:

* [RANZCR 1st Place Soluiton Seg Model (small ver.)](https://www.kaggle.com/haqishen/ranzcr-1st-place-soluiton-seg-model-small-ver)
* [RANZCR 1st Place Soluiton Cls Model (small ver.)](https://www.kaggle.com/haqishen/ranzcr-1st-place-soluiton-cls-model-small-ver)
* [RANZCR 1st Place Soluiton Inference (small ver.)](https://www.kaggle.com/haqishen/ranzcr-1st-place-soluiton-inference-small-ver)

The discussion of the solution is [here](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/226633).

My implementation consists of the following five notebooks. They run on Kaggle CPU, GPU and TPU.

* RANZCR 1st Place Solution by TF (1) Make Masks (this one)
* [RANZCR 1st Place Solution by TF (2) Seg Model](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-2-seg-model)
* [RANZCR 1st Place Solution by TF (3) Gen Masks](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-3-gen-masks)
* [RANZCR 1st Place Solution by TF (4) Cls Model](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-4-cls-model)
* [RANZCR 1st Place Solution by TF (5) Inference](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-5-inference)

I made two more notebooks as compared to the original. The additional ones are for making TFRecord files. They are used to train the segmentation and classification models. Tensorflow can process TFRecord files faster than other types, for example JPEG. For TPU, [here](https://www.kaggle.com/docs/tpu) is useful information.

I made the following four Datasets:

* [RANZCR segmentation masks](https://www.kaggle.com/tt195361/ranzcr-segmentation-masks) -- contains TFRecord files made by this notebook, used to train the segmentation model.
* [RANZCR 1st Place Solution by TF Train Data](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-train-data) -- contains TFRecord files made by the third notebook, used to train the classification model.
* [RANZCR 1st Place Solution by TF Models](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-models) -- contains model weights of the segmentation and classification models, used by the inference model for submission.
* [Segmentation Models Keras](https://www.kaggle.com/tt195361/segmentation-models-keras) -- Python \*.whl files to install [Segmentation Models](https://github.com/qubvel/segmentation_models) without Internet, which is not allowed for submission notebook.

The followings are the points of these notebooks:

* Segmentation model: at first, I tried to convert the original PyTorch model to Keras by torch.onnx.export() and [onnx2keras](https://github.com/nerox8664/onnx2keras). I made a [version](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-2-seg-model?scriptVersionId=57986327), then trained. But it did not went well, accuracy was not improved. So, I changed to use a model in [Segmentation Models](https://github.com/qubvel/segmentation_models).
* 5 channel inputs for classification model: The original one uses 3 channels for image and 2 channels for mask, so total input channels are 5. It is not easy for Keras models to use the pre-trained weights and change the number of input channels at the same time. 
* Data augmentation: the original notebooks use [Albumentations](https://github.com/albumentations-team/albumentations). To train Tensorflow model on CPU/GPU, it's possible to use it as described [here](https://albumentations.ai/docs/examples/tensorflow-example/). However, on TPU, all pipeline needs to be Tensorflow, so I made a similar one by Tensorflow.

The public/private score of the final result is 0.97015 and 0.97090. The original author says that it is sufficient to achieve LB score 0.972.  So, my score is lower than he expected. I used EfficientNetB5 as the base of the segmentation model. I trained only 1 fold for it. For classification, I selected EfficientNetB4 and ran 5 folds.

----
In this first notebook, let's make TFRecords for images, masks, and fold information.
To draw lines and circles for masks, CV2 is used as in the original one. It's not straight forward by Tensorflow.

By using the TFRecord files made in this notebook, I built a dataset [RANZCR segmentation masks](https://www.kaggle.com/tt195361/ranzcr-segmentation-masks). It is used by the segmentation model in the next [notebook](https://www.kaggle.com/tt195361/ranzcr-1st-place-solution-by-tf-2-seg-model).

In [ ]:
DEBUG = False

In [ ]:
# libraries
import os
import numpy as np
import pandas as pd
import ast
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

print(tf.__version__)

## Config

In [ ]:
data_dir = '../input/ranzcr-clip-catheter-line-classification'
image_size = 1024
image_folder = 'train'
jpeg_quality = 100

## Train_v2

train_v2.csv has 2 additional columns, "fold" and "w_anno".

In [ ]:
df_train = pd.read_csv('../input/ranzcr-fold/train_v2.csv')

# If DEBUG == True, use only 500 samples.
df_train = pd.concat([
    df_train.query('fold == 0').sample(100),
    df_train.query('fold == 1').sample(100),
    df_train.query('fold == 2').sample(100),
    df_train.query('fold == 3').sample(100),
    df_train.query('fold == 4').sample(100),
]) if DEBUG else df_train

df_train.shape

df_train['fold'] shows the fold the row belongs to. There are 5 folds and they look evenly distributed for both all the rows and rows with "w_anno == True".

In [ ]:
df_train['fold'].value_counts().sort_index()

In [ ]:
df_train.query('w_anno == True')['fold'].value_counts().sort_index()

In [ ]:
df_train['w_anno'].value_counts().sort_index()

## Annotations and Masks

In [ ]:
df_train_anno = pd.read_csv(os.path.join(data_dir, 'train_annotations.csv'))

print(df_train_anno.shape)

In [ ]:
def read_image(StudyInstanceUID):
    path = os.path.join(data_dir, image_folder, StudyInstanceUID + '.jpg')
    image = cv2.imread(path)[:, :, ::-1]
    return image

In [ ]:
def make_mask(image, StudyInstanceUID):
    df_this = df_train_anno.query(
        f'StudyInstanceUID == "{StudyInstanceUID}"')
    # Use 3 channels for encoding as PNG.
    mask = np.zeros(
        (image.shape[0], image.shape[1], 3)).astype(np.uint8)
    for _, anno in df_this.iterrows():
        anno_this = np.array(ast.literal_eval(anno["data"]))
        mask1 = mask[:, :, 0].copy()
        mask1 = cv2.polylines(
            mask1, np.int32([anno_this]), isClosed=False,
            color=1, thickness=15, lineType=16)
        mask[:, :, 0] = mask1
        mask2 = mask[:, :, 1].copy()
        mask2 = cv2.circle(
            mask2, (anno_this[0][0], anno_this[0][1]),
            radius=15, color=1, thickness=25)
        mask2 = cv2.circle(
            mask2, (anno_this[-1][0], anno_this[-1][1]),
            radius=15, color=1, thickness=25)
        mask[:, :, 1] = mask2

    mask = (mask > 0.5).astype(np.uint8)
    return mask

## Make TFRecord

In [ ]:
def convert_image_mask(image, mask):
    image = cv2.resize(image, (image_size, image_size))
    image = tf.constant(image, dtype=tf.uint8)
    image = tf.image.encode_jpeg(image, quality=jpeg_quality)
    
    mask = cv2.resize(mask, (image_size, image_size))
    mask = tf.constant(mask, dtype=tf.uint8)
    mask = tf.io.encode_png(mask)
    
    return image, mask

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        # BytesList won't unpack a string from an EagerTensor.
        value = value.numpy() 
    elif isinstance(value, str):
        # string needs to be encoded to bytes.
        value = value.encode('utf-8')
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(image, mask, fold):
    feature = {
        'image': _bytes_feature(image),
        'mask': _bytes_feature(mask),
        'fold': _int64_feature(fold),
    }
    
    example_proto = tf.train.Example(
        features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
df_train_w_anno = df_train.query('w_anno==True')
n_df_train_w_anno = len(df_train_w_anno)

print(df_train_w_anno.shape)
print(n_df_train_w_anno)

In [ ]:
# Define a generator to provide a DataFrame row one by one.
def get_next_row():
    for _, row in df_train_w_anno.iterrows():
        yield row

In [ ]:
n_tfrec = 15
n_anno_per_tfrec = (n_df_train_w_anno + n_tfrec - 1) // n_tfrec

n_anno_per_tfrec

In [ ]:
remaining_item_count = len(df_train_w_anno)
tfrec_i = 0
gnr = get_next_row()
while 0 < remaining_item_count:
    tfrec_item_count = min(n_anno_per_tfrec, remaining_item_count)
    tfrec_file_name = "{0:02d}-{1:03d}.tfrec".format(tfrec_i, tfrec_item_count)
    print("Writing {0}...".format(tfrec_file_name))
    with tf.io.TFRecordWriter(tfrec_file_name) as writer:
        for tfrec_item_i in range(tfrec_item_count):
            if tfrec_item_i % 100 == 0:
                print(tfrec_item_i, ", ", end='')
            row = next(gnr)
            image = read_image(row.StudyInstanceUID)
            mask = make_mask(image, row.StudyInstanceUID)
            image, mask = convert_image_mask(image, mask)
            example = serialize_example(image, mask, row.fold)
            writer.write(example)
    print()
    remaining_item_count -= tfrec_item_count
    tfrec_i += 1

In [ ]:
! ls -l

## Verify TFRecords

In [ ]:
def decode_image(image_bytes):
    image = tf.image.decode_jpeg(image_bytes, channels=3)
    return image

def decode_mask(mask_bytes):
    mask = tf.io.decode_png(mask_bytes, channels=3)
    return mask

def read_tfrecord(example):
    TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'mask': tf.io.FixedLenFeature([], tf.string),
        'fold': tf.io.FixedLenFeature([], tf.int64),
    }
    
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    image = decode_image(example['image'])
    mask = decode_mask(example['mask'])
    fold = example['fold']
    return image, mask, fold

def load_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=None)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=None)
    return dataset

In [ ]:
tfrec_file_names = tf.io.gfile.glob('*.tfrec')
ds = load_dataset(tfrec_file_names)

print(ds)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 20,10

f, axarr = plt.subplots(1,5)
masks = []
ds_iter = iter(ds)
for p in range(5):
    img, mask, _ = next(ds_iter)
    axarr[p].imshow(img)
    masks.append(mask)

f, axarr = plt.subplots(1,5)
for p in range(5):
    axarr[p].imshow(masks[p][ : , : , 0])

f, axarr = plt.subplots(1,5)
for p in range(5):
    axarr[p].imshow(masks[p][ : , : , 1])